In [1]:
from __future__ import division
from __future__ import print_function

import numpy as np
from numpy.random import rand
import matplotlib.pyplot as plt
import os

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow.keras as keras

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers

import gc
from numba import cuda
from numba import jit

In [2]:
# This is how the triagle lattice data is generated. You may find it helpful to generate some 
# of your own data
class Ising_tri():
    ''' Simulating the Ising model '''  
    def __init__(self, size, temp):
        self.temp = temp
        self.N = int(size)
    ## monte carlo moves
    
    def mcmove(self, config, N, beta):
        ''' This is to execute the monte carlo moves using 
        Metropolis algorithm such that detailed
        balance condition is satisified'''
        for i in range(N):
            for j in range(N):            
                    a = np.random.randint(0, N) # select a row
                    b = np.random.randint(0, N) # select a column
                    s =  config[a, b] # current state at (a, b)
                    if a%2:
                        nb = config[(a+1)%N,b] +config[(a+1)%N,(b+1)%N] + config[a,(b+1)%N] + \
                        config[(a-1)%N,b] + config[(a-1)%N,(b+1)%N] + config[a,(b-1)%N]
                    else:
                        nb = config[(a+1)%N,b] +config[(a+1)%N,(b-1)%N] + config[a,(b+1)%N] + \
                        config[(a-1)%N,b] + config[(a-1)%N,(b-1)%N] + config[a,(b-1)%N]
                    
                    
                    cost = 2*s*nb
                    if cost < 0:	
                        s *= -1
                    elif rand() < np.exp(-cost*beta):
                        s *= -1
                    config[a, b] = s
        return config
    
    def simulate(self):   
        ''' This module simulates the Ising model'''
        config = 2*np.random.randint(2, size=(self.N,self.N))-1   
        msrmnt = 81
        for i in range(msrmnt):
            self.mcmove(config, self.N, 1.0/self.temp)
        return config

class Ising_sq():
    ''' Simulating the Ising model '''  
    def __init__(self, size, temp):
        self.temp = temp
        self.N = int(size)
    ## monte carlo moves

    def mcmove(self, config, N, beta):
        ''' This is to execute the monte carlo moves using 
        Metropolis algorithm such that detailed
        balance condition is satisified'''
        for i in range(N):
            for j in range(N):            
                    a = np.random.randint(0, N) # select a row
                    b = np.random.randint(0, N) # select a column
                    s =  config[a, b] # current state at (a, b)
                    # periodic boundary condition imposed
                    nb = config[(a+1)%N,b] + config[a,(b+1)%N] + config[(a-1)%N,b] + config[a,(b-1)%N]
                    cost = 2*s*nb
                    if cost < 0:
                        s *= -1
                    elif rand() < np.exp(-cost*beta):
                        s *= -1
                    config[a, b] = s
        return config
    
    def simulate(self):   
        ''' This module simulates the Ising model'''
        config = 2*np.random.randint(2, size=(self.N,self.N))-1   
        times = 100
        for i in range(times):
            self.mcmove(config, self.N, 1.0/self.temp)
        return config

You can import 4-temp data for square and triangular lattices as follows

In [3]:
N = 250
nx, ny = 32, 32

Xsq = np.ndarray((4*N,nx,ny,1))
ysq = np.ndarray(4*N)

for i in np.arange(N):
    Xsq[i + 0*N] = np.loadtxt("./square_T1/square_T1/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ysq[i + 0*N] = 0
    Xsq[i + 1*N] = np.loadtxt("./square_T2/square_T2/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ysq[i + 1*N] = 1
    Xsq[i + 2*N] = np.loadtxt("./square_T3/square_T3/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ysq[i + 2*N] = 2
    Xsq[i + 3*N] = np.loadtxt("./square_T4/square_T4/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ysq[i + 3*N] = 3

Xsq_train, Xsq_test, ysq_train, ysq_test = train_test_split(Xsq, ysq, test_size=0.2, random_state=0)

In [4]:
N = 250
nx, ny = 32, 32

Xtri = np.ndarray((4*N,nx,ny,1))
ytri = np.ndarray(4*N)

for i in np.arange(N):
    Xtri[i + 0*N] = np.loadtxt("./triangle_T1/triangle_T1/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ytri[i + 0*N] = 0
    Xtri[i + 1*N] = np.loadtxt("./triangle_T2/triangle_T2/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ytri[i + 1*N] = 1
    Xtri[i + 2*N] = np.loadtxt("./triangle_T3/triangle_T3/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ytri[i + 2*N] = 2
    Xtri[i + 3*N] = np.loadtxt("./triangle_T4/triangle_T4/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ytri[i + 3*N] = 3

Xtri_train, Xtri_test, ytri_train, ytri_test = train_test_split(Xtri, ytri, test_size=0.2, random_state=0)

Make sure you know the shape of data.

In [5]:
print("Shape of training data:")
print(Xsq_train.shape, Xtri_train.shape)
print(ysq_train.shape, ytri_train.shape)
print("Shape of test data:")
print(Xsq_test.shape, Xtri_test.shape)
print(ysq_test.shape, ytri_test.shape)

Shape of training data:
(800, 32, 32, 1) (800, 32, 32, 1)
(800,) (800,)
Shape of test data:
(200, 32, 32, 1) (200, 32, 32, 1)
(200,) (200,)


### (a) Train a fully connected neural network to do the classification on both datasets. Then, train  a  convolutional  neural  network  to  do  the  classification,  on  both datasets.   Make  a  table  of  your  performance  numbers  for  both  models  and  upload  these  numbers.   This,  together  with  your code,  should be uploaded to the course website when you turn in your homework.

The temperatures for square lattice are $T = 1.5, 2.1, 2.4, 3.5$. $T = 2.5, 3.2, 3.8, 5$ for triangle lattice.


Solution to (a):

In [6]:
class small_FNN:
    def __init__(self):
        model = self
    
    @staticmethod
    def build(width, height, depth, num_classes, channels_first=False):
        model = Sequential()

        if channels_first == False:
            inputShape = (height, width, depth)
            chanDim = -1
        elif channels_first == True:
            inputShape = (depth, height, width)
            chanDim = 1
            
        model.add(Flatten())
        
        model.add(Dense(256,  activation='relu'))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())
        
        model.add(Dense(128,  activation='relu'))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())
        
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())
        
        model.add(Dense(32, activation='relu'))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())
        
        model.add(Dense(16, activation='relu'))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())
        
        model.add(Dense(num_classes, activation="softmax"))
        
        return model

In [7]:
def train_model(input_model, train_data, train_lbls, test_data, 
                test_lbls, num_classes, input_shape, hyperparams):
    # Ensure data is shaped properly, assumes channels last set up
    x_train = train_data
    x_test = test_data
    
    # Create categorical labels
    y_train = keras.utils.to_categorical(train_lbls, num_classes)
    y_test = keras.utils.to_categorical(test_lbls, num_classes)
     
    # Set hyperparameters
    INIT_LR = hyperparams[0]# learning rate
    EPOCHS = hyperparams[1] # number of epochs
    BS = hyperparams[2] # batch size
    OPT = optimizers.Adagrad(lr=INIT_LR) # optimizing function
    
    model = input_model
    
    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer=OPT, metrics=['accuracy'])
    
    
    H = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=EPOCHS,
                  batch_size=BS)
    
    return H, model
    

In [8]:
input_shape = (32, 32, 1)
num_classes = 4
fnn_model_sq = small_FNN.build(width=input_shape[0], height=input_shape[1], depth=input_shape[2],
                   num_classes=num_classes)
fnn_model_tri = small_FNN.build(width=input_shape[0], height=input_shape[1], depth=input_shape[2],
                   num_classes=num_classes)

# hyperparams should be a tuple of: INIT_LR, EPOCHS, BS
FNN_hyperparams = (0.01, 50, 32)
H_sq_FNN, sq_FNN_model = train_model(fnn_model_sq, Xsq_train, ysq_train, Xsq_test, ysq_test, num_classes, input_shape, FNN_hyperparams)
H_tri_FNN, tri_FNN_model = train_model(fnn_model_tri, Xtri_train, ytri_train, Xtri_test, ytri_test, num_classes, input_shape, FNN_hyperparams)

Train on 800 samples, validate on 200 samples
Epoch 1/50
800/800 [==============================] - 1s 1ms/sample - loss: 1.7038 - accuracy: 0.3013 - val_loss: 1.4197 - val_accuracy: 0.2650
Epoch 2/50
800/800 [==============================] - 0s 109us/sample - loss: 1.6188 - accuracy: 0.3200 - val_loss: 1.3636 - val_accuracy: 0.3050
Epoch 3/50
800/800 [==============================] - 0s 112us/sample - loss: 1.4346 - accuracy: 0.3462 - val_loss: 1.3421 - val_accuracy: 0.3100
Epoch 4/50
800/800 [==============================] - 0s 115us/sample - loss: 1.3745 - accuracy: 0.3762 - val_loss: 1.3138 - val_accuracy: 0.3450
Epoch 5/50
800/800 [==============================] - 0s 117us/sample - loss: 1.3112 - accuracy: 0.4075 - val_loss: 1.2906 - val_accuracy: 0.3500
Epoch 6/50
800/800 [==============================] - 0s 115us/sample - loss: 1.2848 - accuracy: 0.4350 - val_loss: 1.2687 - val_accuracy: 0.3700
Epoch 7/50
800/800 [==============================] - 0s 109us/sample - loss: 1.

800/800 [==============================] - 0s 117us/sample - loss: 1.2239 - accuracy: 0.4450 - val_loss: 1.2353 - val_accuracy: 0.4300
Epoch 7/50
800/800 [==============================] - 0s 111us/sample - loss: 1.1924 - accuracy: 0.4325 - val_loss: 1.1871 - val_accuracy: 0.4700
Epoch 8/50
800/800 [==============================] - 0s 103us/sample - loss: 1.1723 - accuracy: 0.4750 - val_loss: 1.1196 - val_accuracy: 0.4750
Epoch 9/50
800/800 [==============================] - 0s 101us/sample - loss: 1.1621 - accuracy: 0.4550 - val_loss: 1.0884 - val_accuracy: 0.4950
Epoch 10/50
800/800 [==============================] - 0s 105us/sample - loss: 1.0999 - accuracy: 0.4950 - val_loss: 1.0564 - val_accuracy: 0.5050
Epoch 11/50
800/800 [==============================] - 0s 109us/sample - loss: 1.0605 - accuracy: 0.5000 - val_loss: 1.0528 - val_accuracy: 0.4900
Epoch 12/50
800/800 [==============================] - 0s 113us/sample - loss: 1.0706 - accuracy: 0.5213 - val_loss: 1.0244 - val_acc

### (b) Train a convolutional neural network to do the classification, on both datasets. Make a table of your performance numbers for (a) and (b). 
Try to optimize the performance of your models and compare the result.

solution to (b):

In [9]:
class SimpleCNN:
    def __init__(self):
        model = self
    
    @staticmethod
    def build(width, height, depth, num_classes, channels_first=False):
        # initialize the model along with the input shape to be
        # "channels last" and the channels dimension itself
        model = Sequential()
        
        if channels_first == False:
            inputShape = (height, width, depth)
            chanDim = -1
        elif channels_first == True:
            inputShape = (depth, height, width)
            chanDim = 1
            
        model.add(Conv2D(32, (5, 5), activation='relu', input_shape=inputShape))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(Dropout(0.25))
                  
        model.add(Flatten())
        model.add(Dropout(0.4))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(num_classes, activation='softmax'))

        # return the constructed network architecture
        return model

In [10]:
input_shape = (32, 32, 1)
num_classes = 4
cnn_model_sq = SimpleCNN.build(width=input_shape[0], height=input_shape[1], depth=input_shape[2],
                   num_classes=num_classes)
cnn_model_tri = SimpleCNN.build(width=input_shape[0], height=input_shape[1], depth=input_shape[2],
                   num_classes=num_classes)

CNN_hyperparams = (0.01, 50, 64)
H_sq_CNN, sq_CNN_model = train_model(cnn_model_sq, Xsq_train, ysq_train, Xsq_test, ysq_test, num_classes, input_shape, CNN_hyperparams)
H_tri_CNN, tri_CNN_model = train_model(cnn_model_tri, Xtri_train, ytri_train, Xtri_test, ytri_test, num_classes, input_shape, CNN_hyperparams)

Train on 800 samples, validate on 200 samples
Epoch 1/50
800/800 [==============================] - 1s 1ms/sample - loss: 1.3770 - accuracy: 0.3025 - val_loss: 1.3355 - val_accuracy: 0.3150
Epoch 2/50
800/800 [==============================] - 0s 79us/sample - loss: 1.3090 - accuracy: 0.3625 - val_loss: 1.2403 - val_accuracy: 0.3950
Epoch 3/50
800/800 [==============================] - 0s 75us/sample - loss: 1.2031 - accuracy: 0.4425 - val_loss: 1.1016 - val_accuracy: 0.5650
Epoch 4/50
800/800 [==============================] - 0s 77us/sample - loss: 1.0361 - accuracy: 0.5288 - val_loss: 0.9425 - val_accuracy: 0.6100
Epoch 5/50
800/800 [==============================] - 0s 77us/sample - loss: 0.8941 - accuracy: 0.5987 - val_loss: 0.9279 - val_accuracy: 0.4950
Epoch 6/50
800/800 [==============================] - 0s 77us/sample - loss: 0.7485 - accuracy: 0.6725 - val_loss: 1.5481 - val_accuracy: 0.2300
Epoch 7/50
800/800 [==============================] - 0s 75us/sample - loss: 0.7559 -

800/800 [==============================] - 0s 82us/sample - loss: 0.7071 - accuracy: 0.6762 - val_loss: 0.9301 - val_accuracy: 0.5000
Epoch 7/50
800/800 [==============================] - 0s 80us/sample - loss: 0.6910 - accuracy: 0.6875 - val_loss: 0.7662 - val_accuracy: 0.5600
Epoch 8/50
800/800 [==============================] - 0s 86us/sample - loss: 0.7228 - accuracy: 0.6675 - val_loss: 0.5507 - val_accuracy: 0.7350
Epoch 9/50
800/800 [==============================] - 0s 91us/sample - loss: 0.5773 - accuracy: 0.7387 - val_loss: 0.6272 - val_accuracy: 0.6900
Epoch 10/50
800/800 [==============================] - 0s 90us/sample - loss: 0.4746 - accuracy: 0.8037 - val_loss: 0.4981 - val_accuracy: 0.7750
Epoch 11/50
800/800 [==============================] - 0s 90us/sample - loss: 0.5948 - accuracy: 0.7212 - val_loss: 0.4055 - val_accuracy: 0.8650
Epoch 12/50
800/800 [==============================] - 0s 90us/sample - loss: 0.4340 - accuracy: 0.8050 - val_loss: 0.4088 - val_accuracy: 

### (c) We have provided a test set of 10 spins configurations for each of the two problems. Each of the spin configurations is not necessarily at the temperatures of the training sets. Calculate your best estimate of the temperatures of these spin configuration. Upload your results to Kaggle.
[Hint: A direct fingerprint of temperature is the distribution of spin up
and down, because you can image that the spins fluctuate more violently
at higher temperature. Although the mothod you use in homework 2 can also work, you may be interested in trying to take distribution into account when you
build the model to estimate temperature and see if you can make use of this extra information. This may help you win the
kaggle. It is totally fine if you find that the information of distribution is not helpful. Note also that a CNN kind-of does this. One possibility is that you may want a CNN that captures enough distribution information.]

Solution to (c)

In [11]:
cwd = str(os.getcwd())
sq_sim_data_path = cwd+"/sq_sim_data.npy"
tri_sim_data_path = cwd+"/tri_sim_data.npy"

temps_vec = np.linspace(0.01,15,num=1500)

"""
sq_sim_data = []
tri_sim_data = []
sim_data_temp = []

for temp in temps_vec:    
    sq_ising_simu = Ising_sq(32, temp)
    tri_ising_simu = Ising_tri(32, temp)
    sq_img = sq_ising_simu.simulate()
    tri_img = tri_ising_simu.simulate()
    
    sq_sim_data.append(sq_img)
    tri_sim_data.append(tri_img)
    
cwd = str(os.getcwd())
sim_data_path_2 = cwd+"/sim_data_2.npy"
sim_data_temp_path_2 = cwd+"/sim_data_temp_2.npy"
sim_data_tot_lv_mean_path = cwd+"/sim_data_tot_lv_mean.npy"

np.save(sq_sim_data_path, np.asarray(sq_sim_data))
np.save(tri_sim_data_path, np.asarray(tri_sim_data))
"""

'\nsq_sim_data = []\ntri_sim_data = []\nsim_data_temp = []\n\nfor temp in temps_vec:    \n    sq_ising_simu = Ising_sq(32, temp)\n    tri_ising_simu = Ising_tri(32, temp)\n    sq_img = sq_ising_simu.simulate()\n    tri_img = tri_ising_simu.simulate()\n    \n    sq_sim_data.append(sq_img)\n    tri_sim_data.append(tri_img)\n    \ncwd = str(os.getcwd())\nsim_data_path_2 = cwd+"/sim_data_2.npy"\nsim_data_temp_path_2 = cwd+"/sim_data_temp_2.npy"\nsim_data_tot_lv_mean_path = cwd+"/sim_data_tot_lv_mean.npy"\n\nnp.save(sq_sim_data_path, np.asarray(sq_sim_data))\nnp.save(tri_sim_data_path, np.asarray(tri_sim_data))\n'

In [12]:
# Get the data ready to be embedded
sq_sim_data = np.load(sq_sim_data_path).reshape(1500,32,32,-1)
tri_sim_data = np.load(tri_sim_data_path).reshape(1500,32,32,-1)

In [13]:
sq_sim_data, tri_sim_data = np.repeat(sq_sim_data, 5, 1), np.repeat(tri_sim_data, 5, 1)
sq_sim_data, tri_sim_data = np.repeat(sq_sim_data, 5, 2), np.repeat(tri_sim_data, 5, 2)
sq_sim_data, tri_sim_data = np.repeat(sq_sim_data, 3, 3), np.repeat(tri_sim_data, 3, 3)

In [14]:
tri_sim_data.shape

(1500, 160, 160, 3)

In [15]:
# Get pre-trained MobileNetV2 
with tf.device('/CPU:0'):
    base_model = tf.keras.applications.MobileNetV2(input_shape=(160,160,3),
                                                   include_top=False,
                                                   weights='imagenet',
                                                   classes=4)
    base_model.trainable = False

In [16]:
# Get the embedded data
global_avg_layer = tf.keras.layers.GlobalAveragePooling2D()

sq_sim_data_emb = global_avg_layer(base_model.predict(sq_sim_data))
sq_sim_data_emb = sq_sim_data_emb.numpy()/sq_sim_data_emb.numpy().max()

tri_sim_data_emb = global_avg_layer(base_model.predict(tri_sim_data))
tri_sim_data_emb = tri_sim_data_emb.numpy()/tri_sim_data_emb.numpy().max()

In [17]:
# Get train/test splits
sq_x_train, sq_x_test, sq_y_train, sq_y_test = train_test_split(sq_sim_data_emb, temps_vec, test_size=0.2, random_state=0)
tri_x_train, tri_x_test, tri_y_train, tri_y_test = train_test_split(tri_sim_data_emb, temps_vec, test_size=0.2, random_state=0)

In [18]:
# Define the model
class RegressionHead:
    def __init__(self):
        model = self
    
    @staticmethod
    def build(channels_first=False):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Dense(612, activation='relu'))
        model.add(tf.keras.layers.Dropout(0.25))
        model.add(tf.keras.layers.Dense(32, activation='relu'))
        model.add(tf.keras.layers.Dropout(0.25))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dense(1, activation='linear'))
        
        return model

In [19]:
INIT_LR = 0.01# learning rate
EPOCHS = 50 # number of epochs
BS = 32 # batch size
OPT = optimizers.Adagrad(lr=INIT_LR) # optimizing function
LOSS = keras.losses.MeanSquaredError()

sq_reg_model = RegressionHead.build()
sq_reg_model.compile(optimizer=OPT, loss=LOSS, metrics=['MeanSquaredError'])
sq_reg_H = sq_reg_model.fit(sq_x_train, sq_y_train, validation_data=(sq_x_test, sq_y_test), epochs=EPOCHS,
                  batch_size=BS)

Train on 1200 samples, validate on 300 samples
Epoch 1/50
1200/1200 [==============================] - 0s 350us/sample - loss: 54.4427 - MeanSquaredError: 54.4427 - val_loss: 46.3159 - val_MeanSquaredError: 46.3159
Epoch 2/50
1200/1200 [==============================] - 0s 76us/sample - loss: 41.7279 - MeanSquaredError: 41.7279 - val_loss: 37.5618 - val_MeanSquaredError: 37.5618
Epoch 3/50
1200/1200 [==============================] - 0s 73us/sample - loss: 32.5192 - MeanSquaredError: 32.5192 - val_loss: 38.8629 - val_MeanSquaredError: 38.8629
Epoch 4/50
1200/1200 [==============================] - 0s 75us/sample - loss: 24.9197 - MeanSquaredError: 24.9197 - val_loss: 21.5944 - val_MeanSquaredError: 21.5944
Epoch 5/50
1200/1200 [==============================] - 0s 76us/sample - loss: 19.4002 - MeanSquaredError: 19.4002 - val_loss: 20.9087 - val_MeanSquaredError: 20.9087
Epoch 6/50
1200/1200 [==============================] - 0s 76us/sample - loss: 15.3215 - MeanSquaredError: 15.3215 - 

1200/1200 [==============================] - 0s 70us/sample - loss: 3.9319 - MeanSquaredError: 3.9319 - val_loss: 2.9962 - val_MeanSquaredError: 2.9962


In [20]:
INIT_LR = 0.01# learning rate
EPOCHS = 50 # number of epochs
BS = 32 # batch size
OPT = optimizers.Adagrad(lr=INIT_LR) # optimizing function
LOSS = keras.losses.MeanSquaredError()

tri_reg_model = RegressionHead.build()
tri_reg_model.compile(optimizer=OPT, loss=LOSS, metrics=['MeanSquaredError'])
tri_reg_H = tri_reg_model.fit(tri_x_train, tri_y_train, validation_data=(tri_x_test, tri_y_test), epochs=EPOCHS,
                  batch_size=BS)

Train on 1200 samples, validate on 300 samples
Epoch 1/50
1200/1200 [==============================] - 0s 361us/sample - loss: 53.4960 - MeanSquaredError: 53.4960 - val_loss: 51.0035 - val_MeanSquaredError: 51.0035
Epoch 2/50
1200/1200 [==============================] - 0s 77us/sample - loss: 40.3129 - MeanSquaredError: 40.3129 - val_loss: 53.8225 - val_MeanSquaredError: 53.8225
Epoch 3/50
1200/1200 [==============================] - 0s 75us/sample - loss: 31.2308 - MeanSquaredError: 31.2308 - val_loss: 38.1518 - val_MeanSquaredError: 38.1518
Epoch 4/50
1200/1200 [==============================] - 0s 76us/sample - loss: 24.0846 - MeanSquaredError: 24.0846 - val_loss: 25.2302 - val_MeanSquaredError: 25.2302
Epoch 5/50
1200/1200 [==============================] - 0s 72us/sample - loss: 18.9162 - MeanSquaredError: 18.9162 - val_loss: 27.5664 - val_MeanSquaredError: 27.5664
Epoch 6/50
1200/1200 [==============================] - 0s 75us/sample - loss: 14.8738 - MeanSquaredError: 14.8738 - 

1200/1200 [==============================] - 0s 72us/sample - loss: 3.0290 - MeanSquaredError: 3.0290 - val_loss: 1.6168 - val_MeanSquaredError: 1.6168


In [37]:
def T_import(dir):
    temp = []
    for i in range(len(os.listdir(dir+'/'))):
        temp.append(np.loadtxt((dir+'/')+str(i).zfill(3), delimiter=','))
    return np.array(temp)

def T10_import(dir):
    temp = []
    for i in range(0,10):
        for j in range(0,10):
            temp.append(np.loadtxt((dir+'/T0'+str(i)+'#')+str(j).zfill(2), delimiter=','))
    return np.array(temp)

In [38]:
tri_t10_img = T10_import(cwd+'/triangle_10T').reshape(100,32,32,-1)
square_t10_img = T10_import(cwd+'/square_10T').reshape(100,32,32,-1)

In [39]:
square_t10_data, tri_t10_data = np.repeat(square_t10_img, 5, 1), np.repeat(tri_t10_img, 5, 1)
square_t10_data, tri_t10_data = np.repeat(square_t10_data, 5, 2), np.repeat(tri_t10_data, 5, 2)
square_t10_data, tri_t10_data = np.repeat(square_t10_data, 3, 3), np.repeat(tri_t10_data, 3, 3)

In [40]:
square_t10_img.shape

(100, 32, 32, 1)

In [41]:
square_t10_data_emb = global_avg_layer(base_model.predict(square_t10_data))
square_t10_data_emb = square_t10_data_emb.numpy()/square_t10_data_emb.numpy().max()

tri_t10_data_emb = global_avg_layer(base_model.predict(tri_t10_data))
tri_t10_data_emb = tri_t10_data_emb.numpy()/tri_t10_data_emb.numpy().max()

In [42]:
tri_reg_model.predict(tri_t10_data_emb)

array([[5.6601267 ],
       [5.2140226 ],
       [6.00695   ],
       [5.011929  ],
       [4.138114  ],
       [4.750653  ],
       [6.067241  ],
       [4.378121  ],
       [4.9823513 ],
       [4.6362586 ],
       [4.02925   ],
       [3.3392138 ],
       [4.7100606 ],
       [3.914775  ],
       [4.248056  ],
       [3.382172  ],
       [3.2406697 ],
       [3.9389744 ],
       [3.1664176 ],
       [4.5941057 ],
       [4.496542  ],
       [4.328278  ],
       [4.00007   ],
       [5.01751   ],
       [4.306549  ],
       [4.09298   ],
       [4.9168344 ],
       [4.5878954 ],
       [4.1558933 ],
       [4.2527957 ],
       [4.22478   ],
       [4.769401  ],
       [3.7035217 ],
       [4.065798  ],
       [4.195915  ],
       [3.9596224 ],
       [3.9306293 ],
       [4.050641  ],
       [4.3738756 ],
       [3.6717181 ],
       [4.711441  ],
       [4.6812267 ],
       [4.0168576 ],
       [4.095689  ],
       [4.2343287 ],
       [4.2828107 ],
       [4.172458  ],
       [4.990

In [43]:
tri_CNN_model.predict_proba(tri_t10_img)

array([[2.65633900e-16, 5.51906005e-08, 8.69663358e-02, 9.13033545e-01],
       [1.51543430e-19, 1.59032631e-11, 4.33319074e-04, 9.99566615e-01],
       [2.51819547e-21, 1.06018200e-12, 9.55850192e-05, 9.99904394e-01],
       [4.15801195e-17, 1.48430834e-09, 4.47533885e-03, 9.95524645e-01],
       [2.39680969e-13, 8.61846218e-07, 3.34095895e-01, 6.65903270e-01],
       [5.23150172e-19, 7.62910290e-11, 1.10261468e-03, 9.98897314e-01],
       [1.13710011e-19, 1.06397356e-11, 6.70520414e-04, 9.99329448e-01],
       [3.12760166e-17, 8.93384122e-09, 2.69313660e-02, 9.73068655e-01],
       [1.68044439e-20, 1.35628774e-11, 3.78648721e-04, 9.99621272e-01],
       [1.79048455e-18, 1.53093399e-10, 2.12737708e-03, 9.97872591e-01],
       [9.51254964e-10, 2.04438320e-03, 9.97881472e-01, 7.40964388e-05],
       [4.68881357e-11, 3.83405160e-04, 9.99500513e-01, 1.16107396e-04],
       [1.98213191e-12, 1.26059003e-05, 8.72999251e-01, 1.26988113e-01],
       [3.48983754e-12, 3.11443982e-05, 9.92305577e

### (d) *Transfer Learning*.  
As we emphasize in class, one can freeze the training of the bottom layers of a network and retrain the top part of the network to adopt to a new situation. Use your CNN that you trained on the squarelattice data to do transfer learning on the triangular lattice data.  How does the performance compare to that of the direct methods?  Add the performance numbers for transfer learning in your table from Part (a). Note that the training time and number of training examples needed for transfer learning is far less than that for the direct  optimization. For  example,  is  50  triangle  example  sufficient  for the re-training process?  Use your transfer learning result to predict the transition temperature of triangle lattice Ising model, as demonstrated in this [Nature Physics](https://www-nature-com.ezp-prod1.hul.harvard.edu/articles/nphys4035.pdf) publication.

As a guideline, you may like to just change the last `Dense` layer with `softmax` activation when you do the transfer learning. Other choices are also OK.

Solution to (d):

In [ ]:
base_model = sq_CNN_model

In [ ]:
base_model.summary()

In [ ]:
base_layers = base_model.layers[0:5]
trainable_layers = [
     Flatten(),
     Dropout(0.25),
     Dense(32, activation='relu'),
     Dense(num_classes, activation='softmax')
]

In [ ]:
trans_model = keras.Sequential(base_layers+trainable_layers)
trans_model.summary()

In [ ]:
input_shape = (32, 32, 1)
num_classes = 4
hyperparams = (0.01, 25, 32)
H_trans, trained_trans_model = train_model(trans_model, Xtri_train, ytri_train, Xtri_test, ytri_test, num_classes, input_shape, hyperparams)